In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import json
#
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
#
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#
import xgboost as xgb

2025-02-22 20:24:13.760954: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 20:24:13.774017: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 20:24:13.892204: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 20:24:14.008296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740227054.104890  115952 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740227054.13

In [3]:
df = pd.read_csv('elongation.csv')
df.head()

,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15,D
0,41.652,24.739,21.857,45.095,36.583,27.420,41.849,29.251,24.4270,39.156,31.525,29.661,22.298,37.114,31.930,122.46
1,26.007,23.648,37.083,21.532,36.033,25.703,38.035,33.229,9.0067,22.466,44.577,13.691,26.670,33.056,30.683,107.44
2,26.297,33.818,21.227,48.211,35.864,39.852,33.195,51.299,36.9970,37.245,37.786,51.839,33.949,46.883,36.635,120.03
3,34.610,14.372,31.512,26.721,22.922,38.254,39.524,33.275,42.9710,20.904,40.671,22.071,17.671,25.432,19.461,109.05
4,37.274,19.442,19.182,48.360,46.518,32.365,27.234,16.937,26.1260,24.334,34.613,39.486,33.299,22.766,34.507,115.82


In [4]:
df['D'].mean()

112.94323513333333

In [3]:
df.shape

(15000, 16)

In [7]:
# features and target
X = df.drop(columns=['D'])
y = df['D']

In [15]:
def build_and_compile_ann(lr):
    model = keras.Sequential([
      layers.Dense(10, activation='relu'), 
      layers.Dense(20, activation='relu'),
      layers.Dense(1)
    ])

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(lr))
    return model

def train_ann(X_train, y_train, epochs, lr):
    ann_model = build_and_compile_ann(lr)
    history = ann_model.fit(
        X_train,
        y_train,
        validation_split=0.2,
        verbose=0, epochs=epochs)
    return ann_model, history

def train_xgb(X_train, y_train):
    xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
    xgb_model.fit(X_train, y_train)
    return xgb_model

In [17]:
metadata = {}

X_df_train, X_df_test, y_df_train, y_df_test = train_test_split(X, y, test_size=0.1)
X_train = X_df_train.values
y_train = y_df_train.values

xgb_model = train_xgb(X_train, y_train)
    
ann_model, history = train_ann(X_train, y_train, epochs=3, lr=0.1)
    
y_pred_ann = ann_model.predict(X_df_test)
y_pred_xgb = xgb_model.predict(X_df_test.values)
r2_ann = r2_score(y_df_test, y_pred_ann.flatten())
mse_ann = mean_squared_error(y_df_test, y_pred_ann.flatten())
r2_xgb = r2_score(y_df_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_df_test, y_pred_xgb)
print(f"R2 score of ANN: ", r2_ann)
print(f"R2 score of XGBoost: ", r2_xgb)
    #
metadata_n = {'ANN_R2': r2_ann,'ANN_MSE':mse_ann, 'XGB_R2': r2_xgb,'XGB_MSE':mse_xgb}
metadata.update(metadata_n)
    #
ann_model.save(f'models/ANN.h5')
xgb_model.save_model(f'models/XGB.json')

with open('models/metadata.json', 'w') as file:
    json.dump(metadata, file)

print("Training complete")

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


R2 score of ANN:  0.911049544644521
R2 score of XGBoost:  0.9777288141871708
Training complete
